In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils



Using TensorFlow backend.


In [3]:
# load ascii text and covert to lowercase
filename = "data/lost-illusions.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [4]:
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

Notice the clean 2-liners to get the set of different characters (usual line for the mapping)

In [6]:
n_chars = len(raw_text)
n_vocab = len(chars)
print(f"Total Characters: ", n_chars)
print(f"Total Vocab: ", n_vocab)

Total Characters:  1408455
Total Vocab:  55


The text is a bit long, we only keep the first 15 000 characters.

In [14]:
character_limit = 15000
raw_text = raw_text[:character_limit]
# create mapping of unique chars to integers
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
n_chars = len(raw_text)
n_vocab = len(chars)
print(f"Total Characters: ", n_chars)
print(f"Total Vocab: ", n_vocab)

Total Characters:  15000
Total Vocab:  46


In [16]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print(f"Total Patterns: ", n_patterns)


Total Patterns:  14900


Note about expected tensor shape for an LSTM network in keras: [samples, time steps, features]
Here the features has not been made yet.  
__The following shaping and one-hot encoding is dirty. See NLP-experiment-1 for a proper way of coding this.__


In [18]:
#FIXME: change the tensor shaping with clean code
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

LSTM definition

In [19]:
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

This model creation is high level keras and will not help that much to create more complicated models.

In [24]:
# define the checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [22]:
model.fit(X, y, epochs=20, batch_size=128, callbacks=callbacks_list)

Epoch 1/20


  128/14900 [..............................] - ETA: 3:46 - loss: 3.8134

  256/14900 [..............................] - ETA: 2:28 - loss: 3.8029

  384/14900 [..............................] - ETA: 1:57 - loss: 3.7906

  512/14900 [>.............................] - ETA: 1:50 - loss: 3.7779

...

14592/14900 [============================>.] - ETA: 1s - loss: 2.7149

14720/14900 [============================>.] - ETA: 0s - loss: 2.7147

14848/14900 [============================>.] - ETA: 0s - loss: 2.7154

14900/14900 [==============================] - 76s 5ms/step - loss: 2.7154



Epoch 00020: loss improved from 2.73180 to 2.71540, saving model to weights-improvement-20-2.7154.hdf5


In [26]:
# load the network weights
filename = "weights-improvement-20-2.7154.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [28]:
#reverse mapping
int_to_char = dict((i, c) for i, c in enumerate(chars))

Generate text

In [33]:
import sys
# pick a random seed
start = numpy.random.randint(0, len(dataX) - 1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")


Seed:
"  was struck. m. le
comte de maucombe, disguised in a provincial printer's jacket, set up,
read, and  "
toe toe to

e tar to

 the ta

r to the t

ar to the ta

...

r to the ta

r to the tar 

to the tar to

 the tar to th

e tar to th
Done.


Model is bad for many reasons, but in particular because the dataset was way too small.   
Nothing to worry about. This was just for training.